In [1]:
#some thing to try to get the maps to show up
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from IPython.display import display

# Import API key
from api_keys import g_key

In [21]:
#read and look at the data
wdata = pd.read_csv("../WeatherPy/output/weather_data.csv")
wdata.drop("Unnamed: 0",axis=1,inplace=True)
wdata

,City,Latitude,Longitude,Temperature (F),Humidity,Wind Speed,Cloud Coverage
0,ushuaia,-54.8000,-68.3000,41.23,92,6.02,100
1,kruisfontein,-34.0033,24.7314,48.18,75,3.96,1
2,itarema,-2.9248,-39.9167,82.81,65,19.80,22
3,sur,22.5667,59.5289,89.37,60,5.37,19
4,oussouye,12.4850,-16.5469,79.59,83,10.51,100
...,...,...,...,...,...,...,...
577,westport,41.1415,-73.3579,72.09,89,5.75,100
578,lorengau,-2.0226,147.2712,80.89,82,12.21,93
579,aykhal,66.0000,111.5000,50.00,87,15.41,100
580,cap malheureux,-19.9842,57.6142,66.47,89,17.40,100


In [5]:
#insert api key into lock
gmaps.configure(api_key=g_key)

In [6]:
#get the locations and humidities of the cities
locations = wdata[["Latitude","Longitude"]].astype(float)
hum = wdata["Humidity"].astype(float)

In [11]:
#make the heatmap
fig = gmaps.figure()

heat = gmaps.heatmap_layer(locations, weights=hum,dissipating=False, max_intensity=100,point_radius = 3)

fig.add_layer(heat)

fig

Figure(layout=FigureLayout(height='420px'))

In [59]:
#find the places with good weather
good_weather_places = wdata.loc[((wdata["Temperature (F)"]>=70) & (wdata["Temperature (F)"]<=80) & (wdata["Wind Speed"]<10) & (wdata["Cloud Coverage"] == 0))]
good_weather_places.reset_index(inplace=True,drop=True)
good_weather_places["Country"] = ["China", "USA","Syria","Tanzania","Romania","Turkey","China","Russia", "USA", "USA","China","France"]
good_weather_places

C:\Users\pauls\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Latitude,Longitude,Temperature (F),Humidity,Wind Speed,Cloud Coverage,Country
0,urumqi,43.8010,87.6005,79.75,34,6.71,0,China
1,laguna,38.4210,-121.4238,76.03,63,5.75,0,USA
2,yabrud,33.9674,36.6574,74.37,38,4.14,0,Syria
3,chake chake,-5.2459,39.7666,72.63,86,8.23,0,Tanzania
4,buzias,45.6492,21.6036,71.33,57,7.72,0,Romania
5,gebze,40.8028,29.4307,74.28,50,4.61,0,Turkey
6,yining,29.0361,114.5636,77.83,84,0.89,0,China
7,divnomorskoye,44.5011,38.1353,74.23,90,5.32,0,Russia
8,superior,46.7208,-92.1041,74.05,55,9.22,0,USA
9,duluth,46.7833,-92.1066,74.03,55,9.22,0,USA


In [63]:
#for each city, find the nearest hotel
hotels = []
for i in range(len(good_weather_places["City"])):
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    coords = f"{good_weather_places['Latitude'][i]},{good_weather_places['Longitude'][i]}"
    params = {"location": coords, "rankby":"distance", "keyword":"hotel", "key":g_key}
    hotel = requests.get(base_url,params=params).json()["results"][0]["name"]
    hotels.append(hotel)
#add the names of hotels to the dataframe
good_weather_places["Hotel"] = hotels

C:\Users\pauls\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [66]:
marker_locations = good_weather_places[["Latitude","Longitude"]]
markers = gmaps.marker_layer(marker_locations,info_box_content=[f'Hotel: {good_weather_places["Hotel"][i]},  City:  {good_weather_places["City"][i]},  Country:  {good_weather_places["Country"][i]}' for i in range(len(good_weather_places["City"]))])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))